In [ ]:
import os
import shutil
import sys
from google.colab import drive

print("Setting up environment...")
drive.mount('/content/drive')

if not os.path.exists('/content/models'):
    print("Restoring Code...")
    !git clone https://github.com/Gabrysse/MLDL2024_project1.git temp_repo
    shutil.copytree('temp_repo/models', '/content/models')
    shutil.rmtree('temp_repo')

zip_path = '/content/drive/MyDrive/semseg/project_data.zip'

if not os.path.exists('/content/dataset/project_data/gta5'):
    print("Restoring Dataset...")
    if not os.path.exists('/content/dataset'): os.makedirs('/content/dataset')

    if os.path.exists(zip_path):
        shutil.unpack_archive(zip_path, '/content/dataset')
        print("Dataset restored.")
    else:
        print(f"Error: Zip file not found at {zip_path}")
else:
    print("Dataset already exists.")

Setting up environment...
Mounted at /content/drive
Restoring Code...
Cloning into 'temp_repo'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 34 (delta 9), reused 3 (delta 3), pack-reused 13 (from 1)
Receiving objects: 100% (34/34), 11.29 KiB | 481.00 KiB/s, done.
Resolving deltas: 100% (9/9), done.
Restoring Dataset...
Dataset restored.


In [ ]:
import os
import sys
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from PIL import Image, ImageFilter
import torchvision.transforms as transforms
import numpy as np
import random
from models.bisenet.build_bisenet import BiSeNet

# --- 1. SETUP ---
print("🚀 Starting Step 3b: Training with Augmentations...")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Ensure models can be imported
if os.path.exists('/content/MLDL2024_project1'):
    sys.path.append('/content/MLDL2024_project1')
elif not os.path.exists('/content/models'):
    # Quick fix if models folder is missing
    !git clone https://github.com/Gabrysse/MLDL2024_project1.git temp_repo
    import shutil
    shutil.copytree('temp_repo/models', '/content/models')

# --- 2. AUGMENTED DATASET CLASS ---
id_mapping = {
    7: 0, 8: 1, 11: 2, 12: 3, 13: 4, 17: 5,
    19: 6, 20: 7, 21: 8, 22: 9, 23: 10, 24: 11, 25: 12,
    26: 13, 27: 14, 28: 15, 31: 16, 32: 17, 33: 18
}

class GTA5AugmentedDataset(Dataset):
    def __init__(self, root_dir, augment=False):
        self.root_dir = root_dir
        self.augment = augment
        self.images_dir = os.path.join(root_dir, 'images')
        self.masks_dir = os.path.join(root_dir, 'labels')
        self.images = []
        self.masks = []

        if os.path.exists(self.images_dir):
            for file_name in sorted(os.listdir(self.images_dir)):
                self.images.append(os.path.join(self.images_dir, file_name))
                self.masks.append(os.path.join(self.masks_dir, file_name))

        # Define Photometric Transforms (Image ONLY)
        self.color_jitter = transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1)
        self.normalize = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self): return len(self.images)

    def __getitem__(self, idx):
        image = Image.open(self.images[idx]).convert('RGB')
        mask = Image.open(self.masks[idx])

        # 1. Base Resize (Project Requirement: 1280x720)
        image = image.resize((1280, 720), Image.BILINEAR)
        mask = mask.resize((1280, 720), Image.NEAREST)

        # --- 2. AUGMENTATIONS ---
        if self.augment:
            # A. Random Horizontal Flip (Geometric - Applied to BOTH)
            if random.random() > 0.5:
                image = image.transpose(Image.FLIP_LEFT_RIGHT)
                mask = mask.transpose(Image.FLIP_LEFT_RIGHT)

            # B. Color Jitter (Photometric - Image ONLY)
            if random.random() > 0.5:
                image = self.color_jitter(image)

            # C. Gaussian Blur (Photometric - Image ONLY)
            if random.random() > 0.5:
                image = image.filter(ImageFilter.GaussianBlur(radius=random.uniform(0.1, 2.0)))

        # 3. Final Formatting
        image = self.normalize(image)

        mask_np = np.array(mask)
        target_mask = np.full(mask_np.shape, 255, dtype=np.uint8)
        for k, v in id_mapping.items():
            target_mask[mask_np == k] = v

        return image, torch.from_numpy(target_mask).long()

# --- 3. TRAINING SETUP ---
# Important: Create a NEW checkpoint file so we don't overwrite your baseline!
checkpoint_path = '/content/drive/MyDrive/bisenet_gta5_aug_checkpoint.pth'

model = BiSeNet(num_classes=19, context_path='resnet18')
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss(ignore_index=255)

GTA_PATH = '/content/dataset/project_data/gta5'

if os.path.exists(GTA_PATH):
    # Enable augmentations here!
    train_dataset = GTA5AugmentedDataset(GTA_PATH, augment=True)
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)

    print(f"🎬 Starting Training (Augmented)...")
    print(f"💾 Saving to: {checkpoint_path}")

    start_epoch = 0
    num_epochs = 50 # [cite: 27]

    # Resume capability for this new checkpoint
    if os.path.exists(checkpoint_path):
        print("🔄 Resuming augmented training...")
        checkpoint = torch.load(checkpoint_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1

    for epoch in range(start_epoch, num_epochs):
        model.train()
        for i, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)

            if isinstance(outputs, tuple):
                loss = criterion(outputs[0], labels) + 0.1 * criterion(outputs[1], labels) + 0.1 * criterion(outputs[2], labels)
            else:
                loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            if i % 50 == 0:
                print(f"Epoch [{epoch+1}/{num_epochs}] Step [{i}/{len(train_loader)}] Loss: {loss.item():.4f}")

        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()
        }, checkpoint_path)
        print(f"✅ Epoch {epoch+1} Saved.")
else:
    print("❌ GTA5 dataset not found. Please unzip project_data.zip again.")

🚀 Starting Step 3b: Training with Augmentations...
🎬 Starting Training (Augmented)...
💾 Saving to: /content/drive/MyDrive/bisenet_gta5_aug_checkpoint.pth
Epoch [1/50] Step [0/313] Loss: 4.3948
Epoch [1/50] Step [50/313] Loss: 1.1009
Epoch [1/50] Step [100/313] Loss: 0.8189
Epoch [1/50] Step [150/313] Loss: 0.6185
Epoch [1/50] Step [200/313] Loss: 0.6791
Epoch [1/50] Step [250/313] Loss: 0.5196
Epoch [1/50] Step [300/313] Loss: 0.5247
✅ Epoch 1 Saved.
Epoch [2/50] Step [0/313] Loss: 0.5069
Epoch [2/50] Step [50/313] Loss: 0.5373
Epoch [2/50] Step [100/313] Loss: 0.4711
Epoch [2/50] Step [150/313] Loss: 0.4364
Epoch [2/50] Step [200/313] Loss: 0.4240
Epoch [2/50] Step [250/313] Loss: 0.4166
Epoch [2/50] Step [300/313] Loss: 0.4238
✅ Epoch 2 Saved.
Epoch [3/50] Step [0/313] Loss: 0.4025
Epoch [3/50] Step [50/313] Loss: 0.4116
Epoch [3/50] Step [100/313] Loss: 0.3659
Epoch [3/50] Step [150/313] Loss: 0.3188
Epoch [3/50] Step [200/313] Loss: 0.4270
Epoch [3/50] Step [250/313] Loss: 0.3799
E

In [ ]:
import os
import shutil
import sys
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from PIL import Image, ImageFilter
import torchvision.transforms as transforms
import numpy as np
import random
from google.colab import drive

# --- 1. MOUNT DRIVE & RESTORE ENVIRONMENT ---
print("🚀 Initiating Auto-Resume Sequence...")
drive.mount('/content/drive')

# A. Restore Code
if not os.path.exists('/content/models'):
    print("⚠️ Code folder missing. Restoring...")
    !git clone https://github.com/Gabrysse/MLDL2024_project1.git temp_repo
    shutil.copytree('temp_repo/models', '/content/models')
    shutil.rmtree('temp_repo')
    print("✅ Code restored to /content/models")

# B. Restore Dataset
if not os.path.exists('/content/dataset/project_data/gta5'):
    print("⚠️ Dataset missing. Unzipping again... (This takes ~2 mins)")
    if not os.path.exists('/content/dataset'): os.makedirs('/content/dataset')

    # Path to your zip file (Adjust if you moved it)
    zip_path = '/content/drive/MyDrive/semseg/project_data.zip'

    if os.path.exists(zip_path):
        shutil.unpack_archive(zip_path, '/content/dataset')
        print("✅ Dataset restored!")
    else:
        print(f"❌ CRITICAL: Could not find zip at {zip_path}")

from models.bisenet.build_bisenet import BiSeNet

# --- 2. DEFINE AUGMENTED DATASET ---
# We must redefine this class so the script can run standalone
id_mapping = {
    7: 0, 8: 1, 11: 2, 12: 3, 13: 4, 17: 5,
    19: 6, 20: 7, 21: 8, 22: 9, 23: 10, 24: 11, 25: 12,
    26: 13, 27: 14, 28: 15, 31: 16, 32: 17, 33: 18
}

class GTA5AugmentedDataset(Dataset):
    def __init__(self, root_dir, augment=False):
        self.root_dir = root_dir
        self.augment = augment
        self.images_dir = os.path.join(root_dir, 'images')
        self.masks_dir = os.path.join(root_dir, 'labels')
        self.images = sorted(os.listdir(self.images_dir))
        self.color_jitter = transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1)
        self.normalize = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self): return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.images_dir, self.images[idx])
        mask_path = os.path.join(self.masks_dir, self.images[idx])
        image = Image.open(img_path).convert('RGB').resize((1280, 720), Image.BILINEAR)
        mask = Image.open(mask_path).resize((1280, 720), Image.NEAREST)

        if self.augment:
            # Aug 1: Flip (Geometric)
            if random.random() > 0.5:
                image = image.transpose(Image.FLIP_LEFT_RIGHT)
                mask = mask.transpose(Image.FLIP_LEFT_RIGHT)

            # Aug 2: Color/Blur (Photometric)
            if random.random() > 0.5:
                image = self.color_jitter(image)
            if random.random() > 0.5:
                image = image.filter(ImageFilter.GaussianBlur(radius=random.uniform(0.1, 2.0)))

        image = self.normalize(image)
        mask_np = np.array(mask)
        target_mask = np.full(mask_np.shape, 255, dtype=np.uint8)
        for k, v in id_mapping.items(): target_mask[mask_np == k] = v
        return image, torch.from_numpy(target_mask).long()

# --- 3. RESUME TRAINING ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"⚙️ Device: {device}")

# THIS IS THE CHECKPOINT WE WANT TO RESUME
checkpoint_path = '/content/drive/MyDrive/bisenet_gta5_aug_checkpoint.pth'

model = BiSeNet(num_classes=19, context_path='resnet18')
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss(ignore_index=255)

start_epoch = 0
num_epochs = 50

if os.path.exists(checkpoint_path):
    print(f"🔄 Found Checkpoint: {checkpoint_path}")
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    print(f"⏩ Resuming from Epoch {start_epoch}")
else:
    print("🆕 No checkpoint found. Starting from scratch (Epoch 0)!")

# Setup Loader
GTA_PATH = '/content/dataset/project_data/gta5'
if os.path.exists(GTA_PATH):
    train_dataset = GTA5AugmentedDataset(GTA_PATH, augment=True)
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)

    print("🎬 Action: Training Resumed!")
    for epoch in range(start_epoch, num_epochs):
        model.train()
        for i, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)

            if isinstance(outputs, tuple):
                loss = criterion(outputs[0], labels) + 0.1 * criterion(outputs[1], labels) + 0.1 * criterion(outputs[2], labels)
            else:
                loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            if i % 50 == 0:
                print(f"Epoch [{epoch+1}/{num_epochs}] Step [{i}/{len(train_loader)}] Loss: {loss.item():.4f}")

        # Save every epoch
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()
        }, checkpoint_path)
        print(f"✅ Epoch {epoch+1} Saved.")
else:
    print("❌ Dataset path not valid. Unzip failed?")

🚀 Initiating Auto-Resume Sequence...
Mounted at /content/drive
⚠️ Code folder missing. Restoring...
Cloning into 'temp_repo'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (18/18), done.
Receiving objects: 100% (34/34), 11.29 KiB | 11.29 MiB/s, done.
Resolving deltas: 100% (9/9), done.
remote: Total 34 (delta 9), reused 3 (delta 3), pack-reused 13 (from 1)
✅ Code restored to /content/models
⚠️ Dataset missing. Unzipping again... (This takes ~2 mins)
✅ Dataset restored!
⚙️ Device: cuda
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 150MB/s]


Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth


100%|██████████| 171M/171M [00:01<00:00, 116MB/s]


🔄 Found Checkpoint: /content/drive/MyDrive/bisenet_gta5_aug_checkpoint.pth
⏩ Resuming from Epoch 9
🎬 Action: Training Resumed!
Epoch [10/50] Step [0/313] Loss: 0.2370
Epoch [10/50] Step [50/313] Loss: 0.2997
Epoch [10/50] Step [100/313] Loss: 0.2620
Epoch [10/50] Step [150/313] Loss: 0.3085
Epoch [10/50] Step [200/313] Loss: 0.2116
Epoch [10/50] Step [250/313] Loss: 0.2866
Epoch [10/50] Step [300/313] Loss: 0.2285
✅ Epoch 10 Saved.
Epoch [11/50] Step [0/313] Loss: 0.2464
Epoch [11/50] Step [50/313] Loss: 0.2484
Epoch [11/50] Step [100/313] Loss: 0.2632
Epoch [11/50] Step [150/313] Loss: 0.2352
Epoch [11/50] Step [200/313] Loss: 0.2257
Epoch [11/50] Step [250/313] Loss: 0.2542
Epoch [11/50] Step [300/313] Loss: 0.2487
✅ Epoch 11 Saved.
Epoch [12/50] Step [0/313] Loss: 0.2879
Epoch [12/50] Step [50/313] Loss: 0.2414
Epoch [12/50] Step [100/313] Loss: 0.2211
Epoch [12/50] Step [150/313] Loss: 0.2776
Epoch [12/50] Step [200/313] Loss: 0.2714
Epoch [12/50] Step [250/313] Loss: 0.2300
Epoch 

In [ ]:
import os
import shutil
import sys
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from PIL import Image, ImageFilter
import torchvision.transforms as transforms
import numpy as np
import random
from google.colab import drive

# --- 1. MOUNT DRIVE & RESTORE ENVIRONMENT ---
print("🚀 Initiating Auto-Resume Sequence...")
drive.mount('/content/drive')

# A. Restore Code
if not os.path.exists('/content/models'):
    print("⚠️ Code folder missing. Restoring...")
    !git clone https://github.com/Gabrysse/MLDL2024_project1.git temp_repo
    shutil.copytree('temp_repo/models', '/content/models')
    shutil.rmtree('temp_repo')
    print("✅ Code restored to /content/models")

# B. Restore Dataset
if not os.path.exists('/content/dataset/project_data/gta5'):
    print("⚠️ Dataset missing. Unzipping again... (This takes ~2 mins)")
    if not os.path.exists('/content/dataset'): os.makedirs('/content/dataset')

    # Path to your zip file (Adjust if you moved it)
    zip_path = '/content/drive/MyDrive/semseg/project_data.zip'

    if os.path.exists(zip_path):
        shutil.unpack_archive(zip_path, '/content/dataset')
        print("✅ Dataset restored!")
    else:
        print(f"❌ CRITICAL: Could not find zip at {zip_path}")

from models.bisenet.build_bisenet import BiSeNet

# --- 2. DEFINE AUGMENTED DATASET ---
# We must redefine this class so the script can run standalone
id_mapping = {
    7: 0, 8: 1, 11: 2, 12: 3, 13: 4, 17: 5,
    19: 6, 20: 7, 21: 8, 22: 9, 23: 10, 24: 11, 25: 12,
    26: 13, 27: 14, 28: 15, 31: 16, 32: 17, 33: 18
}

class GTA5AugmentedDataset(Dataset):
    def __init__(self, root_dir, augment=False):
        self.root_dir = root_dir
        self.augment = augment
        self.images_dir = os.path.join(root_dir, 'images')
        self.masks_dir = os.path.join(root_dir, 'labels')
        self.images = sorted(os.listdir(self.images_dir))
        self.color_jitter = transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1)
        self.normalize = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self): return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.images_dir, self.images[idx])
        mask_path = os.path.join(self.masks_dir, self.images[idx])
        image = Image.open(img_path).convert('RGB').resize((1280, 720), Image.BILINEAR)
        mask = Image.open(mask_path).resize((1280, 720), Image.NEAREST)

        if self.augment:
            # Aug 1: Flip (Geometric)
            if random.random() > 0.5:
                image = image.transpose(Image.FLIP_LEFT_RIGHT)
                mask = mask.transpose(Image.FLIP_LEFT_RIGHT)

            # Aug 2: Color/Blur (Photometric)
            if random.random() > 0.5:
                image = self.color_jitter(image)
            if random.random() > 0.5:
                image = image.filter(ImageFilter.GaussianBlur(radius=random.uniform(0.1, 2.0)))

        image = self.normalize(image)
        mask_np = np.array(mask)
        target_mask = np.full(mask_np.shape, 255, dtype=np.uint8)
        for k, v in id_mapping.items(): target_mask[mask_np == k] = v
        return image, torch.from_numpy(target_mask).long()

# --- 3. RESUME TRAINING ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"⚙️ Device: {device}")

# THIS IS THE CHECKPOINT WE WANT TO RESUME
checkpoint_path = '/content/drive/MyDrive/bisenet_gta5_aug_checkpoint.pth'

model = BiSeNet(num_classes=19, context_path='resnet18')
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss(ignore_index=255)

start_epoch = 0
num_epochs = 50

if os.path.exists(checkpoint_path):
    print(f"🔄 Found Checkpoint: {checkpoint_path}")
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    print(f"⏩ Resuming from Epoch {start_epoch}")
else:
    print("🆕 No checkpoint found. Starting from scratch (Epoch 0)!")

# Setup Loader
GTA_PATH = '/content/dataset/project_data/gta5'
if os.path.exists(GTA_PATH):
    train_dataset = GTA5AugmentedDataset(GTA_PATH, augment=True)
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)

    print("🎬 Action: Training Resumed!")
    for epoch in range(start_epoch, num_epochs):
        model.train()
        for i, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)

            if isinstance(outputs, tuple):
                loss = criterion(outputs[0], labels) + 0.1 * criterion(outputs[1], labels) + 0.1 * criterion(outputs[2], labels)
            else:
                loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            if i % 50 == 0:
                print(f"Epoch [{epoch+1}/{num_epochs}] Step [{i}/{len(train_loader)}] Loss: {loss.item():.4f}")

        # Save every epoch
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()
        }, checkpoint_path)
        print(f"✅ Epoch {epoch+1} Saved.")
else:
    print("❌ Dataset path not valid. Unzip failed?")

🚀 Initiating Auto-Resume Sequence...
Mounted at /content/drive
⚠️ Code folder missing. Restoring...
Cloning into 'temp_repo'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 34 (delta 9), reused 3 (delta 3), pack-reused 13 (from 1)
Receiving objects: 100% (34/34), 11.29 KiB | 11.29 MiB/s, done.
Resolving deltas: 100% (9/9), done.
✅ Code restored to /content/models
⚠️ Dataset missing. Unzipping again... (This takes ~2 mins)
✅ Dataset restored!
⚙️ Device: cuda
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 56.6MB/s]


Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth


100%|██████████| 171M/171M [00:01<00:00, 129MB/s]


🔄 Found Checkpoint: /content/drive/MyDrive/bisenet_gta5_aug_checkpoint.pth
⏩ Resuming from Epoch 37
🎬 Action: Training Resumed!
Epoch [38/50] Step [0/313] Loss: 0.1682
Epoch [38/50] Step [50/313] Loss: 0.1686
Epoch [38/50] Step [100/313] Loss: 0.2043
Epoch [38/50] Step [150/313] Loss: 0.1913
Epoch [38/50] Step [200/313] Loss: 0.1645
Epoch [38/50] Step [250/313] Loss: 0.1737
Epoch [38/50] Step [300/313] Loss: 0.1880
✅ Epoch 38 Saved.
Epoch [39/50] Step [0/313] Loss: 0.1575
Epoch [39/50] Step [50/313] Loss: 0.2051
Epoch [39/50] Step [100/313] Loss: 0.2305
Epoch [39/50] Step [150/313] Loss: 0.2143
Epoch [39/50] Step [200/313] Loss: 0.1748
Epoch [39/50] Step [250/313] Loss: 0.1766
Epoch [39/50] Step [300/313] Loss: 0.1600
✅ Epoch 39 Saved.
Epoch [40/50] Step [0/313] Loss: 0.1773
Epoch [40/50] Step [50/313] Loss: 0.1778
Epoch [40/50] Step [100/313] Loss: 0.2187
Epoch [40/50] Step [150/313] Loss: 0.1967
Epoch [40/50] Step [200/313] Loss: 0.1832
Epoch [40/50] Step [250/313] Loss: 0.2076
Epoch

In [ ]:
import torch
import os
import sys
import numpy as np
from torch.utils.data import DataLoader, Dataset
from models.bisenet.build_bisenet import BiSeNet
from PIL import Image
from tqdm import tqdm
import torchvision.transforms as transforms

# --- CONFIGURATION ---
CITYSCAPES_PATH = '/content/dataset/project_data/cityscapes'
# ✅ TESTING: Combined Augmentation Model
CHECKPOINT_PATH = '/content/drive/MyDrive/bisenet_gta5_aug_checkpoint.pth'
NUM_CLASSES = 19
BATCH_SIZE = 4
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Class Names in correct order
CLASSES = [
    "Road", "Sidewalk", "Building", "Wall", "Fence", "Pole",
    "Traffic Light", "Traffic Sign", "Vegetation", "Terrain", "Sky",
    "Person", "Rider", "Car", "Truck", "Bus", "Train", "Motorcycle", "Bicycle"
]

if os.path.exists('/content/MLDL2024_project1'): sys.path.append('/content/MLDL2024_project1')

# --- DATASET ---
class CityscapesDataset(Dataset):
    def __init__(self, root, split='val', transform=None):
        self.root = root
        self.transform = transform
        self.images_dir = os.path.join(root, 'leftImg8bit', split)
        self.masks_dir = os.path.join(root, 'gtFine', split)
        self.images = []
        self.masks = []

        if os.path.exists(self.images_dir):
            for city in sorted(os.listdir(self.images_dir)):
                img_dir_path = os.path.join(self.images_dir, city)
                mask_dir_path = os.path.join(self.masks_dir, city)
                if not os.path.isdir(img_dir_path): continue
                for file_name in sorted(os.listdir(img_dir_path)):
                    if file_name.endswith('_leftImg8bit.png'):
                        self.images.append(os.path.join(img_dir_path, file_name))
                        mask_name = file_name.replace('_leftImg8bit.png', '_gtFine_labelTrainIds.png')
                        self.masks.append(os.path.join(mask_dir_path, mask_name))

    def __len__(self): return len(self.images)
    def __getitem__(self, idx):
        image = Image.open(self.images[idx]).convert('RGB').resize((1024, 512), Image.BILINEAR)
        mask = Image.open(self.masks[idx]).resize((1024, 512), Image.NEAREST)
        if self.transform: image = self.transform(image)
        return image, torch.from_numpy(np.array(mask)).long()

# --- EVALUATION ---
print(f"🚀 detailed Evaluation for: {CHECKPOINT_PATH}")

model = BiSeNet(num_classes=NUM_CLASSES, context_path='resnet18')
model.to(DEVICE)

if os.path.exists(CHECKPOINT_PATH):
    checkpoint = torch.load(CHECKPOINT_PATH, map_location=DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    print("✅ Model loaded.")
else:
    print("❌ Checkpoint not found.")
    exit()

model.eval()
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

dataset = CityscapesDataset(CITYSCAPES_PATH, split='val', transform=transform)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

hist = np.zeros((NUM_CLASSES, NUM_CLASSES))

print("Processing...")
with torch.no_grad():
    for images, labels in tqdm(dataloader):
        images = images.to(DEVICE)
        labels = labels.numpy()
        output = model(images)
        if isinstance(output, tuple): output = output[0]
        preds = torch.argmax(output, dim=1).cpu().numpy()

        mask = (labels >= 0) & (labels < NUM_CLASSES)
        hist += np.bincount(
            NUM_CLASSES * labels[mask].astype(int) + preds[mask],
            minlength=NUM_CLASSES ** 2
        ).reshape(NUM_CLASSES, NUM_CLASSES)

# --- CALCULATE & PRINT RESULTS ---
iou = np.diag(hist) / (hist.sum(axis=1) + hist.sum(axis=0) - np.diag(hist))
miou = np.nanmean(iou)

print("\n" + "="*40)
print(f"🏆 Final mIoU: {miou * 100:.2f}%")
print("="*40)
print("📝 Per-Class IoU (Copy these to Table 4):")
print("-" * 40)
for i, class_name in enumerate(CLASSES):
    print(f"{class_name:15s}: {iou[i] * 100:.2f}%")
print("-" * 40)

🚀 detailed Evaluation for: /content/drive/MyDrive/bisenet_gta5_aug_checkpoint.pth
✅ Model loaded.
Processing...


100%|██████████| 125/125 [00:59<00:00,  2.11it/s]


🏆 Final mIoU: 29.97%
📝 Per-Class IoU (Copy these to Table 4):
----------------------------------------
Road           : 71.81%
Sidewalk       : 19.36%
Building       : 74.84%
Wall           : 26.07%
Fence          : 14.29%
Pole           : 24.28%
Traffic Light  : 24.98%
Traffic Sign   : 16.16%
Vegetation     : 79.58%
Terrain        : 23.30%
Sky            : 78.03%
Person         : 40.45%
Rider          : 3.82%
Car            : 50.56%
Truck          : 5.90%
Bus            : 3.24%
Train          : 4.86%
Motorcycle     : 6.70%
Bicycle        : 1.17%
----------------------------------------
